In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



import pandas as pd

url = r"https://www.forexfactory.com/calendar?month=this"
def ff_scrape_raw(url, useful_events, currency=["USD"], year=2024, columns=["Date", "Currency", "Event", "Actual", "Forecast", "Previous"]):
    # Initialize web driver and open the page
    driver = webdriver.Chrome()

    # Wait for the page elements to load
    driver.implicitly_wait(10)

    try:
        # Get the table
        driver.get(url)
        
        table = driver.find_element(By.CLASS_NAME, "calendar__table")
        
        # Initialize empty array
        data = []
        
        # Iterate over each table row
        for row in table.find_elements(By.TAG_NAME, "tr"):
            # list comprehension to get each cell's data and filter out empty cells
            row_data = list([td.text for td in row.find_elements(By.TAG_NAME, "td")])
            print(row_data)
            
            if row_data == [] or row_data == [""] or row_data == ["", " "]:
                continue
            
            # elif len(row_data) == 2:
            #     date_str = row_data[0][4:]
            #     full_date_str = f"{date_str} {year}"
            #     date = pd.to_datetime(full_date_str, format='%b %d %Y')
            #     data.append([date] + [None] * 5)
            
            elif len(row_data) == 11:
                date_str = row_data[0][4:]
                full_date_str = f"{date_str} {year}"
                date = pd.to_datetime(full_date_str, format='%b %d %Y')
                
                for idx, value in enumerate(row_data[:][7:10]):
                    if "%" in value:
                        row_data[7 + idx] = float(value[0:-1])/100
                    
                    
                data.append([date] + [row_data[3]] + [row_data[5]] + [i for i in row_data[7:10]])
                
            elif len(row_data) == 10:
                for idx, value in enumerate(row_data[:][6:9]):
                    if "%" in value:
                        row_data[6 + idx] = float(value[0:-1])/100
                        
                data.append([date] + [row_data[2]] + [row_data[4]] + [i for i in row_data[6:9]])

        df = pd.DataFrame(data, columns=columns)
        df.set_index("Date", inplace=True)
        
        # Filter currency and useful events
        # df = df.query(f"Currency in {currency} and Event in {useful_events}")
            
    except Exception as e:
        print(e)
    finally:
        driver.quit()
        
    return df
    
    
useful_events = ["Federal Funds Rate", "Manufacturing PMI", "ISM Manufacturing PMI", "Flash Manufacturing PMI", "Core PPI m/m", "PPI m/m",  "CPI m/m", "Core CPI m/m", "Core PCE Price Index m/m", "GDP m/m", "Unemployment Rate", "ADP Non-Farm Employment Change", "Non-Farm Employment Change"]
print(ff_scrape_raw(url, useful_events)[30:45])

[]
[]
['']
['']
['Fri\nMar 1', '12:45am', '', 'NZD', '', 'Building Consents m/m', '', '-8.8%', '', '3.6%', '']
['2:30am', '', 'JPY', '', 'Unemployment Rate', '', '2.4%', '2.4%', '2.4%', '']
['3:05am', '', 'NZD', '', 'RBNZ Gov Orr Speaks', '', '', '', '', '']
['3:30am', '', 'JPY', '', 'Final Manufacturing PMI', '', '47.2', '47.2', '47.2', '']
['4:10am', '', 'USD', '', 'FOMC Member Williams Speaks', '', '', '', '', '']
['4:30am', '', 'CNY', '', 'Manufacturing PMI', '', '49.1', '49.1', '49.2', '']
['', '', 'CNY', '', 'Non-Manufacturing PMI', '', '51.4', '50.9', '50.7', '']
['4:45am', '', 'CNY', '', 'Caixin Manufacturing PMI', '', '50.9', '50.7', '50.8', '']
['8:00am', '', 'JPY', '', 'Consumer Confidence', '', '39.1', '38.4', '38.0', '']
['8:30am', '', 'AUD', '', 'Commodity Prices y/y', '', '-11.0%', '', '-10.4%', '']
['10:00am', '', 'GBP', '', 'Nationwide HPI m/m', '', '0.7%', '0.2%', '0.7%', '']
['10:30am', '', 'CHF', '', 'Retail Sales y/y', '', '0.3%', '0.4%', '-0.1%', '']
['11:15am', '